# Predicting utilization of medical services: achieving equalized coverage

This tutorial demonstrates how to use the equalized coverage software package [1]. We also show that usual (marginal) conformal methods do not attain equal coverage across the two groups under study: non-white and white individuals. Then, we compare the performance of joint vs. groupwise model fitting and show that, in this example, the former yields shorter predictive intervals. Follow the instructions in `cqr/get_meps_data/` folder to download the data set.

Please refer to `prediction_bias_example.ipynb` for more details about this data set. Also, please check the tutorials `cqr_real_data_example.ipynb` and `cqr_synthetic_data_example.ipynb` that illustrate how to construct marginal prediction intervals using CQR and conformal prediction.

[1] Y. Romano, R. F. Barber, C. Sabbatti and E. J. Candès. "With malice towards none: Assessing uncertainty via equalized coverage." 2019.

## MEPS data set

The Medical Expenditure Panel Survey (MEPS) 2016 data set, provided by the Agency for Healthcare Research and Quality, contains information on individuals and their utilization of medical services. The features used for modeling include age, marital status, race, poverty status, functional limitations, health status, health insurance type, and more. The goal is to predict the health care system utilization of each individual; a score that reflects the number of visits to a doctor’s office, hospital visits, etc. After removing observations with missing entries, there are $n=15656$ observations on $p=139$ features. We set the sensitive attribute $A$ to \textbf{race}, with $A=0$ for non-white and $A=1$ for white individuals, resulting in $n_0=9640$ samples for the first group and $n_1=6016$ for the second.

## Equalized Coverage

Let $\{(X_i, A_i, Y_i)\}$, $i = 1, \ldots, n$, be some training data where the vector $X_i \in \mathbb{R}^p$ may contain the sensitive attribute $A_i \in \{0,1,2,\dots\}$ as one of the features. Imagine we hold a test point with known $X_{n+1}$ and $A_{n+1}$ and aim to construct a prediction interval $C(X_{n+1},A_{n+1}) \subseteq \mathbb{R}$ which contains the unknown response $Y_{n+1} \in \mathbb{R}$ with probability at least $1-\alpha$ on average within each group; here, $0 < 1-\alpha < 1$ is a desired coverage level. Formally, we assume that the training and test samples $ \{(X_i,A_i,Y_i)\}_{i=1}^{n+1} $ are drawn exchangeably from some arbitrary and unknown distribution $P_{XAY}$, and we wish that our  prediction interval obeys the following property
$$\mathbb{P}\{Y_{n+1} \in C(X_{n+1},A_{n+1}) \mid A_{n+1}=a\} \geq 1-\alpha$$
for all $a$. The probability is taken over the $n$ training samples and the test case. 
The probability statement above must hold for any distribution $P_{XAY}$, sample size $n$, and regardless of the group identifier $A_{n+1}$. (While this only ensures that coverage is at least $1-\alpha$ for each group---and, therefore, the groups may have unequal coverage level---we show in our paper that under mild conditions the coverage can also be upper bounded to lie very close to the target level $1-\alpha$.)

## Constructing prediction intervals

In this experiment, we set the miscoverage rate to 10%.

First, let's randomly split the data into train (80%) and test (20%) sets.

In [1]:
import os
import sys
import torch
import random
import numpy as np
np.warnings.filterwarnings('ignore')

from cqr import helper
from datasets import datasets
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from nonconformist.nc import RegressorNc
from nonconformist.nc import SignErrorErrFunc
from nonconformist.nc import QuantileRegAsymmetricErrFunc


# set some arbitrary seed for reproducability 
seed = 30

random_state_train_test = seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    
dataset_base_path = '/Users/romano/mydata/regression_data/'

dataset_name = "meps_21"

# used to determine the size of test set
test_ratio = 0.2

# load the dataset
X, y = datasets.GetDataset(dataset_name, dataset_base_path)

# display basic information
print("Data set name: %s" % (dataset_name))
print("Data set dimensions: (n=%d, p=%d)" % 
      (X.shape[0], X.shape[1]))

print("A=0 : non-white")
print("A=1 : white")
print("Samples per group: A=0 (n=%d) ; A=1 (n=%d)" % 
      (X[X[:,-1]==0].shape[0], X[X[:,-1]==1].shape[0]))

dataset_name_group_0 = dataset_name + "_non_white"
dataset_name_group_1 = dataset_name + "_white"

# divide the dataset into test and train based on the test_ratio parameter
x_train_orig, x_test_orig, y_train_orig, y_test_orig = train_test_split(X,
                                                                        y,
                                                                        test_size=test_ratio,
                                                                        random_state=random_state_train_test)

# compute input dimensions
n_train = x_train_orig.shape[0]
in_shape = x_train_orig.shape[1]

Data set name: meps_21
Data set dimensions: (n=15656, p=139)
A=0 : non-white
A=1 : white
Samples per group: A=0 (n=9640) ; A=1 (n=6016)


### Split and standardize

Our method starts by randomly splitting the $n$ training points into two disjoint subsets; a proper training set $ \left\lbrace (X_i,A_i,Y_i): i \in {I}_1 \right\rbrace  $ and a calibration set $ \left\lbrace (X_i,A_i,Y_i): i \in {I}_2 \right\rbrace  $. We also standardize the features to have zero mean and unit variance; the means and variances are computed using the proper training examples. We transform the response variable by $Y = \log(1 + (\text{utilization score}))$ as the raw score is highly skewed.

In [2]:
# divide the data into proper training set and calibration set
idx = np.random.permutation(n_train)
n_half = int(np.floor(n_train/2))
idx_train, idx_cal = idx[:n_half], idx[n_half:2*n_half]

# zero mean and unit variance scaling 
scalerX = StandardScaler()
scalerX = scalerX.fit(x_train_orig[idx_train])
x_train = scalerX.transform(x_train_orig)
x_test = scalerX.transform(x_test_orig)

# scale the response as it is highly skewed
y_train = np.log(1.0 + y_train_orig)
y_test = np.log(1.0 + y_test_orig)
# reshape the response
y_train = np.squeeze(np.asarray(y_train))
y_test = np.squeeze(np.asarray(y_test))

### Conformal Neural Network

Below, we define the parameters of a classic regression algorithm, minimizing the mean-squared-error loss. The function is formulated as a network network. 

In [3]:
# desired miscoverage level
alpha = 0.1

# desired quantile levels, used when fitting a quantile neural network model
quantiles_net = [alpha/2, 1-alpha/2]

# pytorch's optimizer object
nn_learn_func = torch.optim.Adam

# number of epochs
epochs = 1000

# learning rate
lr = 0.0005

# mini-batch size
batch_size = 64

# hidden dimension of the network
hidden_size = 64

# dropout regularization rate
dropout = 0.1

# weight decay regularization
wd = 1e-6

# seed for splitting the data in cross-validation.
cv_test_ratio = 0.1

# ratio of held-out data, used in cross-validation
cv_random_state = 1

Below, we define the function `condition` that extracts the group identifier from a given feature vector $X$. Here, the race attribute is the last feature in $X$.

In [4]:
# function that extracts the group identifier
def condition(x, y=None):
    return int(x[0][-1]>0)

# extract groups
category_map = np.array([condition((x_test[i, :], None)) for i in range(x_test.shape[0])])
categories = np.unique(category_map)

### Marginal Conformal Neural Network

Below, we run marginal split conformal prediction (which is not guarenteed to attain valid group-conditional coverage). First, we fit a neural network to the proper training examples. Then, we use conformal inference to construct marginal prediction intervals.

In [5]:
model = helper.MSENet_RegressorAdapter(model=None,
                                       fit_params=None,
                                       in_shape = in_shape,
                                       hidden_size = hidden_size,
                                       learn_func = nn_learn_func,
                                       epochs = epochs,
                                       batch_size=batch_size,
                                       dropout=dropout,
                                       lr=lr,
                                       wd=wd,
                                       test_ratio=cv_test_ratio,
                                       random_state=cv_random_state)
        
nc = RegressorNc(model, SignErrorErrFunc())

y_lower, y_upper = helper.run_icp(nc, x_train, y_train, x_test, idx_train, idx_cal, alpha)

method_name = "Marginal Conformal Neural Network"

# compute and print average coverage and average length
coverage_sample, length_sample = helper.compute_coverage_per_sample(y_test,
                                                                    y_lower,
                                                                    y_upper,
                                                                    alpha,
                                                                    method_name,
                                                                    x_test,
                                                                    condition)

Marginal Conformal Neural Network: Group 0 : Percentage in the range (expecting 90.00): 92.546584
Marginal Conformal Neural Network: Group 0 : Average length: 2.928768
Marginal Conformal Neural Network: Group 1 : Percentage in the range (expecting 90.00): 88.083333
Marginal Conformal Neural Network: Group 1 : Average length: 2.928768


As can be seen, in this example, the marginal approach undercovers group 1 (white individuals) and overcovers group 0 (non-white individuals).

### Group-Conditional Conformal Neural Network ( joint )

We now turn to run the group-conditional variant. This method trains one neural netowok model for both group.

In [6]:
model = helper.MSENet_RegressorAdapter(model=None,
                                       fit_params=None,
                                       in_shape = in_shape,
                                       hidden_size = hidden_size,
                                       learn_func = nn_learn_func,
                                       epochs = epochs,
                                       batch_size=batch_size,
                                       dropout=dropout,
                                       lr=lr,
                                       wd=wd,
                                       test_ratio=cv_test_ratio,
                                       random_state=cv_random_state)
nc = RegressorNc(model, SignErrorErrFunc())

y_lower, y_upper = helper.run_icp(nc, x_train, y_train, x_test, idx_train, idx_cal, alpha, condition)

method_name = "Conditional Conformal Neural Network (joint)"

# compute and print average coverage and average length
coverage_sample, length_sample = helper.compute_coverage_per_sample(y_test,
                                                                    y_lower,
                                                                    y_upper,
                                                                    alpha,
                                                                    method_name,
                                                                    x_test,
                                                                    condition)

Conditional Conformal Neural Network (joint): Group 0 : Percentage in the range (expecting 90.00): 91.407867
Conditional Conformal Neural Network (joint): Group 0 : Average length: 2.799576
Conditional Conformal Neural Network (joint): Group 1 : Percentage in the range (expecting 90.00): 90.166667
Conditional Conformal Neural Network (joint): Group 1 : Average length: 3.130488


For this realization, the coverage is valid for both groups.

### Group-Conditional Conformal Neural Network ( groupwise )

It is intersting to compare the performace of the method above---joint training---that fits one model to the whole proper training set, to another approach---groupwise training---that fits a separate model per group (one for non-white and the second for white individuals).

In [7]:
estimator_list = []
nc_list = []
for i in range(len(categories)):

    # define a NN model per group
    estimator_list.append(helper.MSENet_RegressorAdapter(model=None,
                                                         fit_params=None,
                                                         in_shape = in_shape,
                                                         hidden_size = hidden_size,
                                                         learn_func = nn_learn_func,
                                                         epochs = epochs,
                                                         batch_size=batch_size,
                                                         dropout=dropout,
                                                         lr=lr,
                                                         wd=wd,
                                                         test_ratio=cv_test_ratio,
                                                         random_state=cv_random_state))

    # append the model to the list
    nc_list.append(RegressorNc(estimator_list[i], SignErrorErrFunc()))

# run groupswise learning procedure
y_lower, y_upper = helper.run_icp_sep(nc_list, x_train, y_train, x_test, idx_train, idx_cal, alpha, condition)

method_name = "Conditional Conformal Neural Network (groupwise)"

# compute and print average coverage and average length
coverage_sample, length_sample = helper.compute_coverage_per_sample(y_test,
                                                                    y_lower,
                                                                    y_upper,
                                                                    alpha,
                                                                    method_name,
                                                                    x_test,
                                                                    condition)

Conditional Conformal Neural Network (groupwise): Group 0 : Percentage in the range (expecting 90.00): 91.666667
Conditional Conformal Neural Network (groupwise): Group 0 : Average length: 2.813798
Conditional Conformal Neural Network (groupwise): Group 1 : Percentage in the range (expecting 90.00): 90.833333
Conditional Conformal Neural Network (groupwise): Group 1 : Average length: 3.172685


### Marginal CQR Neural Network

We now turn to demonstrate the performace of conformalized quantile regression. Starting with the marginal approach, we fit a quantile neural network model and set the low and high quantile to 5% and 95%. 

In [8]:
# define quantile neural network model
quantile_estimator = helper.AllQNet_RegressorAdapter(model=None,
                                                     fit_params=None,
                                                     in_shape=in_shape,
                                                     hidden_size=hidden_size,
                                                     quantiles=quantiles_net,
                                                     learn_func=nn_learn_func,
                                                     epochs=epochs,
                                                     batch_size=batch_size,
                                                     dropout=dropout,
                                                     lr=lr,
                                                     wd=wd,
                                                     test_ratio=cv_test_ratio,
                                                     random_state=cv_random_state,
                                                     use_rearrangement=False)
# define the CQR object
nc = RegressorNc(quantile_estimator, QuantileRegAsymmetricErrFunc())

# run CQR procedure
y_lower, y_upper = helper.run_icp(nc, x_train, y_train, x_test, idx_train, idx_cal, alpha)

method_name = "Marginal CQR Neural Network"

# compute and print average coverage and average length
coverage_sample, length_sample = helper.compute_coverage_per_sample(y_test,
                                                                    y_lower,
                                                                    y_upper,
                                                                    alpha,
                                                                    method_name,
                                                                    x_test,
                                                                    condition)

Marginal CQR Neural Network: Group 0 : Percentage in the range (expecting 90.00): 91.356108
Marginal CQR Neural Network: Group 0 : Average length: 2.490780
Marginal CQR Neural Network: Group 1 : Percentage in the range (expecting 90.00): 90.750000
Marginal CQR Neural Network: Group 1 : Average length: 3.172656


Notice that, for this realization, we obtain valid coverage for both groups, although the marginal method is not designed to give such a guarentee.

### Group-Conditional CQR Neural Network ( joint )

Moving to the group-conditional variant of CQR, we use the join training approach and fit one model for both groups.  

In [9]:
quantile_estimator = helper.AllQNet_RegressorAdapter(model=None,
                                                             fit_params=None,
                                                             in_shape=in_shape,
                                                             hidden_size=hidden_size,
                                                             quantiles=quantiles_net,
                                                             learn_func=nn_learn_func,
                                                             epochs=epochs,
                                                             batch_size=batch_size,
                                                             dropout=dropout,
                                                             lr=lr,
                                                             wd=wd,
                                                             test_ratio=cv_test_ratio,
                                                             random_state=cv_random_state,
                                                             use_rearrangement=False)
                
# define the CQR object
nc = RegressorNc(quantile_estimator, QuantileRegAsymmetricErrFunc())

# run CQR procedure
y_lower, y_upper = helper.run_icp(nc, x_train, y_train, x_test, idx_train, idx_cal, alpha, condition)

method_name = "Conditional CQR Neural Network (joint)"

# compute and print average coverage and average length
coverage_sample, length_sample = helper.compute_coverage_per_sample(y_test,
                                                                    y_lower,
                                                                    y_upper,
                                                                    alpha,
                                                                    method_name,
                                                                    x_test,
                                                                    condition)

Conditional CQR Neural Network (joint): Group 0 : Percentage in the range (expecting 90.00): 91.200828
Conditional CQR Neural Network (joint): Group 0 : Average length: 2.619198
Conditional CQR Neural Network (joint): Group 1 : Percentage in the range (expecting 90.00): 90.083333
Conditional CQR Neural Network (joint): Group 1 : Average length: 3.064932


The coverage is around the desired level. 

### Group-Conditional CQR Neural Network ( groupwise )

Now, let's check the groupwise traning alternative.

In [10]:
quantile_estimator_list = []
nc_list = []
for i in range(len(categories)):
    quantile_estimator_list.append(helper.AllQNet_RegressorAdapter(model=None,
                                                     fit_params=None,
                                                     in_shape=in_shape,
                                                     hidden_size=hidden_size,
                                                     quantiles=quantiles_net,
                                                     learn_func=nn_learn_func,
                                                     epochs=epochs,
                                                     batch_size=batch_size,
                                                     dropout=dropout,
                                                     lr=lr,
                                                     wd=wd,
                                                     test_ratio=cv_test_ratio,
                                                     random_state=cv_random_state,
                                                     use_rearrangement=False))      

    # append CQR object
    nc_list.append(RegressorNc(quantile_estimator_list[i], QuantileRegAsymmetricErrFunc()))

# run CQR procedure
y_lower, y_upper = helper.run_icp_sep(nc_list, x_train, y_train, x_test, idx_train, idx_cal, alpha, condition)

method_name = "Conditional CQR Neural Network (groupwise)"

# compute and print average coverage and average length
coverage_sample, length_sample = helper.compute_coverage_per_sample(y_test,
                                                                    y_lower,
                                                                    y_upper,
                                                                    alpha,
                                                                    method_name,
                                                                    x_test,
                                                                    condition)

Conditional CQR Neural Network (groupwise): Group 0 : Percentage in the range (expecting 90.00): 91.614907
Conditional CQR Neural Network (groupwise): Group 0 : Average length: 2.648615
Conditional CQR Neural Network (groupwise): Group 1 : Percentage in the range (expecting 90.00): 89.666667
Conditional CQR Neural Network (groupwise): Group 1 : Average length: 3.086279


The average coverage (of this realization) is around the nominal level.

## Summary

In this experiment we illustrate how the coverage rates of the conditional methods are close to the target level. In our paper, we repeat this experiment for 40 different train-test splits and average the results. Then, the obtained coverage per group is exactly 90%. Moreover, the difference between joint and groupwise training becomes apparent, suggesting that for this data set the joint approach is more effective.